# Practise Pandas and clean merge data

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/merge_16.csv")
# print(file.columns.tolist())
# print(file["Extension (P)"][4])
# file.dtypes()
# print(file.head())
# für array nur von den Daten: file.values()
# file.sort_index(axis = 1, ascending = True) //um überschriften zu sortieren zb nach alphabet
#sorted = file.sort_values(by = "Extension (P)")
# print(a["Extension (P)"])
# für statistics also mean std usw: file.describe()
# print(file[0:3])
# mehrere spalten auswählen: file.loc[:, ['A','B']]


# a= df['Extension (P)'].values
# print(a)
# print(np.any(np.isnan(a)))

df = df.sort_values(by = "Backspin Rate (P)")
print(df["Backspin Rate (P)"])
for i in range(2289,2269, -1):
    df = df.drop(df.index[[i]])
print(df["Backspin Rate (P)"])

In [ ]:
koord = df.iloc[:,df.columns.get_loc("0"):df.columns.get_loc("159")]

"""clean data by deleting nan columns"""
for col in df.columns.tolist():
    if (type(df[col][0]) is not np.float64 and type(df[col][0]) is not np.int64) or np.any(np.isnan(df[col].values)):
        df.drop(col, axis=1, inplace=True)

for col in koord.columns.tolist():
    df[col]=koord[col]

print(df.columns.tolist())

In [ ]:
df.to_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/merge_clean.csv")

In [ ]:
cl = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/merge_clean.csv")
#print(cl.columns.tolist())
#print(cl["0"])


In [ ]:
cf = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/cf_data.csv")
sv = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/sv_data.csv")

# Get coordinate values and pitch type labels seperately

In [ ]:
coordinates = cf.iloc[:,cf.columns.get_loc("0"):cf.columns.get_loc("166")]

In [ ]:
unique  = np.unique(cf["Pitch Type"].values)

In [ ]:
def oneHot(dataframe, column):
    unique  = np.unique(dataframe[column].values)
    l = len(dataframe.index)
    loc = dataframe.columns.get_loc(column)
    labels = np.zeros((l, 12))
    for i in range(l):
        #print(cf.iloc[i,loc])
        pitch = dataframe.iloc[i,loc]
        ind = unique.tolist().index(pitch)
        labels[i, ind] = 1
    return labels, unique

labels, _ = oneHot(cf, "Pitch Type")

In [ ]:
data_array = coordinates.values

# Change pandas dataframe to np.array
(Problem: missing values in den Koordinaten)

In [ ]:
M, N = data_array.shape
data = np.zeros((M,N,18,2))

for i in range(M):
    for j in range(N):
        if not pd.isnull(data_array[i,j]):
            data[i,j]=np.array(eval(data_array[i,j]))

#print(data)

In [ ]:
missing = pd.isnull(data_array)
print("Ratio missing: ", np.count_nonzero(missing)/(M*N))

# Tensorflow conv net

In [ ]:
train_x = data[:13000,:]
test_x = data[13000:, :]
train_t= labels[:13000,:]
test_t = labels[13000:, :]

print(len(train_x))
print(len(test_x))

In [ ]:
import tensorflow as tf

x = tf.placeholder(tf.float32, (None, N, 18, 2), name = "input")
x_ = tf.reshape(x, (-1, N, 36 ,1))
y = tf.placeholder(tf.float32, (None, len(labels[0])))

net = tf.layers.conv2d(x_, filters=16, kernel_size=5, strides=2, activation=tf.nn.relu)
net = tf.layers.conv2d(net, filters=16, kernel_size=3, strides=1, activation=tf.nn.relu)
net = tf.layers.conv2d(net, filters=32, kernel_size=3, strides=1, activation=tf.nn.relu)
net = tf.layers.conv2d(net, filters=1, kernel_size=1)
shapes = net.get_shape().as_list()
ff = tf.reshape(net, (-1, shapes[1]*shapes[2]))
ff = tf.layers.dense(ff, 128, activation = tf.nn.relu)
ff = tf.layers.dense(ff, len(labels[0]), activation = tf.nn.sigmoid)

loss = tf.reduce_mean(tf.square(y-ff))
optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss)

In [ ]:
def batches(x, y, batchsize=32):
    permute = np.random.permutation(len(x))
    for i in range(0, len(x)-batchsize, batchsize):
        indices = permute[i:i+batchsize]
        yield x[indices], y[indices]
    
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Run session for 2000 epochs
for epoch in range(2000 + 1):
    for batch_x, batch_t in batches(train_x, train_t, 32):
        sess.run(optimizer, {x: batch_x, y: batch_t})
    print(epoch, sess.run(loss, {x: test_x, y: test_t}))
    

In [ ]:
print(sess.run(loss, {x: test_x, y: test_t}))

# one hot encoding

In [ ]:
def decode_one_hot(results):
    """takes the maximum value and gets the corresponding pitch type
    input: array of size trials * pitchTypesNr
    returns: array of size trials containing the pitch type as a string
    """
    unique  = np.unique(cf["Pitch Type"].values)
    p = []
    for pitch, i in enumerate(results):
        ind = np.argmax(pitch)
        p.append(unique[ind])
    return p

testing = sess.run(ff, {x: test_x, y: test_t})
pitches = decode_one_hot(testing)
label_pitches = (cf["Pitch Type"].values)[13000:]

# evaluate error rate
right = 0
wrong = 0
for i in range(len(pitches)):
    if pitches[i]==label_pitches[i]:
        right+=1
    else:
        wrong+=1
print(right/(right+wrong))

In [ ]:
def oneHot(dataframe, column):
    unique  = np.unique(dataframe[column].values)
    l = len(dataframe.index)
    loc = dataframe.columns.get_loc(column)
    labels = np.zeros((l, 12))
    for i in range(l):
        #print(cf.iloc[i,loc])
        pitch = dataframe.iloc[i,loc]
        ind = unique.tolist().index(pitch)
        labels[i, ind] = 1
    return labels, unique

def decode_one_hot(results, unique):
    """takes the maximum value and gets the corresponding pitch type
    input: array of size trials * pitchTypesNr
    returns: array of size trials containing the pitch type as a string
    """
    #unique  = np.unique(cf["Pitch Type"].values)
    p = []
    print(results)
    for i, pitch in enumerate(results):
        #print(pitch)
        ind = np.argmax(pitch)
        #print(ind)
        p.append(unique[ind])
    return p

labels, unique = oneHot(frame, "Pitch Type")

new = decode_one_hot(labels, unique)
print(np.sum(new==frame["Pitch Type"]))

# Only pitcher 

In [ ]:
df = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/cf_data.csv")
#df = df.sort_values(by = "Player") # Ball Picther or Strike outcome pitcher
# bis 8615 ball/pitcher
pitcher = df.loc[frame["Player"]=="Pitcher"]
pitcher.to_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/cf_only_pitcher.csv")

In [ ]:
frame = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/cf_only_pitcher.csv")
print(len(frame["Player"]))

In [ ]:
a = frame.loc[frame["Player"]=="Pitcher"]

In [ ]:
print(a.iloc[650, a.columns.get_loc("Player")])

In [ ]:
arr1 = (["hi", "ni", "gu"])
arr2 = (["hi", "nina", "gu"])
arr3 = np.asarray(arr1)==np.asarray(arr2)
print(arr3)
print(np.sum(arr3))

print(type(frame["Player"].values[6000:]))

In [ ]:
cf = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/cf_only_pitcher.csv")

print("csv eingelesen")
# COORDINATES
coordinates = cf.iloc[:,cf.columns.get_loc("0"):cf.columns.get_loc("166")]
data_array = coordinates.values
M, N = data_array.shape
data = np.zeros((M,N,18,2))

c=0
for i in range(M):
    for j in range(N):
        if not pd.isnull(data_array[i,j]):
            data[i,j]=np.array(eval(data_array[i,j]))
        else:
            c+=1

np.save("coord_array", data)

In [ ]:
SEP = 6000
EPOCHS = 30

print("imports")
cf = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/cf_only_pitcher.csv")

print("csv eingelesen")
# COORDINATES
"""
coordinates = cf.iloc[:,cf.columns.get_loc("0"):cf.columns.get_loc("166")]
data_array = coordinates.values
M, N = data_array.shape
data = np.zeros((M,N,18,2))

c=0
for i in range(M):
    for j in range(N):
        if not pd.isnull(data_array[i,j]):
            data[i,j]=np.array(eval(data_array[i,j]))
        else:
            c+=1
print("nan in daten:",c)
"""

data = np.load("coord_array.npy")
M,N, _,_ = data.shape

# LABELS
def oneHot(dataframe, column):
    unique  = np.unique(dataframe[column].values)
    l = len(dataframe.index)
    loc = dataframe.columns.get_loc(column)
    labels = np.zeros((l, 12))
    for i in range(l):
        #print(cf.iloc[i,loc])
        pitch = dataframe.iloc[i,loc]
        ind = unique.tolist().index(pitch)
        labels[i, ind] = 1
    return labels, unique

def decode_one_hot(results, unique):
    """takes the maximum value and gets the corresponding pitch type
    input: array of size trials * pitchTypesNr
    returns: array of size trials containing the pitch type as a string
    """
    #unique  = np.unique(cf["Pitch Type"].values)
    p = []
    for _, pitch in enumerate(results):
        ind = np.argmax(pitch)
        p.append(unique[ind])
    return p

labels, unique = oneHot(cf, "Pitch Type")
label_pitches = (cf["Pitch Type"].values)[SEP:]

# NET

np.save("labels", labels)
np.save("unique", unique)
np.save("label_pitches", label_pitches)


In [ ]:
df = pd.read_csv("/Users/ninawiedemann/Desktop/UNI/Praktikum/merge_16.csv")
print(df.columns.tolist())

In [ ]:
df = df.loc[df["Player"] == "Pitcher"]
coordina = df.iloc[:, df.columns.get_loc("0"):df.columns.get_loc("159")]

In [ ]:
M, N = coordina.values.shape
SEP = int(M*0.9)
print("The training data will be up to %d and the test data from %d to %d"%(SEP, SEP, M))

In [ ]:
pointer = df.columns.get_loc("0")
columns = df.columns.tolist()
start = pointer
while(True):
    try:
        zahl = int(columns[pointer])
        pointer+=1
    except ValueError:
        break

coord = df.iloc[:, start:pointer]

# Standardizing and balancing

In [ ]:
import pandas as pd
import numpy as np
cf = pd.read_csv("merge_16.csv")
cf = cf[cf["Player"]=="Pitcher"]

In [ ]:
import scipy as sp
import scipy.stats

print(np.unique(cf["Pitch Type"].values))

types = cf["Pitch Type"].values
note_frequency = sp.stats.itemfreq(types)
print(note_frequency)
smaller_20 = (note_frequency[np.where(note_frequency[:,1]<30)])[:,0].flatten()

for typ in smaller_20:
    cf = cf.drop(cf[cf["Pitch Type"]==typ].index)


print(np.unique(cf["Pitch Type"].values))
#cf = cf.drop(cf[cf["Pitch Type"]=='Fastball (Split-finger)' or cf["Pitch Type"]=="Unknown Pitch Type"].index)

In [ ]:
arr = np.load("coord_array.npy")
print(np.any(np.isnan(arr)))
means = np.mean(arr, axis = 1)
std = np.std(arr, axis = 1)
res = np.asarray([(arr[:,i]-means)/(std+0.0001) for i in range(len(arr[0]))])
new = np.swapaxes(res, 0,1)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

data = compute_class_weight("auto", np.unique(cf["Pitch Type"].values),cf["Pitch Type"].values )

In [ ]:
labels_string= cf["Pitch Type"].values
unique =np.unique(labels_string)
#print(np.unique(pitches))
#print(np.sum(data))
nr_classes=len(unique)
ex_per_class = 4

"""liste=np.zeros((7,4))
for i, types in enumerate(np.unique(pitches)):
    liste[i] = (np.random.choice(np.where(pitches==types)[0], 4))
print(liste)
"""
index_liste = []
for pitches in unique:
    index_liste.append(np.where(labels_string==pitches))
    
def balanced_batches(y):   
    for j in range(5):
        liste=np.zeros((nr_classes, ex_per_class))
        for i in range(nr_classes):
            liste[i] = np.random.choice(index_liste[i][0], ex_per_class)
        #print(liste.flatten().astype(int))
        yield y[liste.flatten().astype(int)]

In [ ]:
for batch_x in balanced_batches(labels_string):
    print(batch_x)

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
#from sklearn.preprocessing import StandardScaler
from data_preprocess import Preprocessor

def decode_one_hot(results, unique):
    """takes the maximum value and gets the corresponding pitch type
    input: array of size trials * pitchTypesNr
    returns: array of size trials containing the pitch type as a string
    """
    #unique  = np.unique(cf["Pitch Type"].values)
    p = []
    for _, pitch in enumerate(results):
        ind = np.argmax(pitch)
        if pitch[ind]>0.5:
            p.append(unique[ind])
        else:
            p.append("Too small")
    return p

leaky_relu = lambda x: tf.maximum(0.2*x, x)


ex_per_class = 4
EPOCHS = 10
PATH = "/Users/ninawiedemann/Desktop/UNI/Praktikum/sv_data.csv"
LABELS = "Pitch Type"
act = leaky_relu
CUT_OFF_Classes = 60

prepro = Preprocessor(PATH, CUT_OFF_Classes)
data = np.load("coord_sv.npy") #prepro.get_coord_arr("coord_sv.npy")

M,N,nr_joints,_ = data.shape
SEP = int(M*0.9)

labels, unique = prepro.get_labels_onehot(LABELS)
labels_string = prepro.get_labels(LABELS)
#labels_test = decode_one_hot(labels[SEP:, :], unique)

nr_classes = len(np.unique(labels_string))
BATCHSIZE = nr_classes*ex_per_class
print("nr classes", nr_classes, "Batchsize", BATCHSIZE)

# NET

ind = np.random.permutation(len(data))
train_ind = ind[:SEP]
test_ind = ind[SEP:]

train_x = data[train_ind]
test_x = data[test_ind]
train_t= labels[train_ind]
test_t = labels[test_ind]
labels_string_train = labels_string[train_ind]
labels_string_test = labels_string[test_ind]

"""
DATA TESTING:
indiuh = np.where(ind==2000)
print("Labels nach preprocc von 2000", labels_string[2000])
print("new Index of 2000", indiuh, "test ob where funkt: ", ind[indiuh])
print("train coord of 2000 u 140", train_x[indiuh, 140])
print("labels_string von 2000", labels_string_train[indiuh])
print("one hot von 2000", train_t[indiuh])
"""

index_liste = []
for pitches in unique:
    index_liste.append(np.where(labels_string_train==pitches))

len_test = len(test_x)
len_train = len(train_x)
print("Test set size: ", len_test, " train set size: ", len_train)
print("Shapes of train_x", train_x.shape, "shape of test_x", test_x.shape)

/Users/ninawiedemann/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,

csv eingelesen with length  9082
Only Pitcher rows
[['Changeup' 363]
 ['Curveball' 428]
 ['Eephus' 1]
 ['Fastball (2-seam)' 960]
 ['Fastball (4-seam)' 1451]
 ['Fastball (Cut)' 191]
 ['Fastball (Split-finger)' 33]
 ['Knuckle curve' 54]
 ['Knuckleball' 214]
 ['Sinker' 181]
 ['Slider' 662]
 ['Unknown Pitch Type' 3]]
Removed because not enought class members:  ['Eephus' 'Fastball (Split-finger)' 'Knuckle curve' 'Unknown Pitch Type']
nr classes 8 Batchsize 32
Test set size:  445  train set size:  4005
Shapes of train_x (4005, 249, 18, 2) shape of test_x (445, 249, 18, 2)


# normal conv net

In [ ]:
x = tf.placeholder(tf.float32, (None, N, nr_joints, 2), name = "input")
x_ = tf.reshape(x, (-1, N, nr_joints*2))
y = tf.placeholder(tf.float32, (None, len(labels[0])))

net = tf.layers.conv1d(x_, filters=256, kernel_size=5, strides=2, activation=act)
net = tf.layers.conv1d(net, filters=256, kernel_size=3, strides=1, activation=act)
net = tf.layers.conv1d(net, filters=128, kernel_size=3, strides=1, activation=act)
net = tf.layers.conv1d(net, filters=1, kernel_size=1, activation = act)
shapes = net.get_shape().as_list()
ff = tf.reshape(net, (-1, shapes[1]*shapes[2]))
ff = tf.layers.dense(ff, 1024, activation = act)
ff = tf.layers.dense(ff, 128, activation = act)
logits = tf.layers.dense(ff, len(labels[0]), activation = None)
out = tf.nn.softmax(logits)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits)) # tf.reduce_mean(tf.square(y-ff))
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

# basic lstm

In [17]:
import tensorflow as tf
from tensorflow.contrib import rnn

learning_rate = 0.001
training_iters = 100000
batch_size = BATCHSIZE
display_step = 10

tf.reset_default_graph()

# Network Parameters
n_input = nr_joints*2 # MNIST data input (img shape: 28*28)
n_steps = N # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 12 # MNIST total classes (0-9 digits)

# tf Graph input
x_ = tf.placeholder("float", [None, n_steps, nr_joints, 2])
x = tf.reshape(x_, (-1, N, nr_joints*2))
y = tf.placeholder("float", [None, n_classes])

"""# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}"""


def RNN(x):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)
    
    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    """with tf.variable_scope("myrnn") as scope:
        for i in range(n_steps-1):
            if i > 0:
                scope.reuse_variables()
            output, state = lstm_cell(x[i], state)"""

    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.layers.dense(outputs[-1], n_classes)   #tf.matmul(outputs[-1], weights['out']) + biases['out']

out_logits = RNN(x)
out = tf.nn.softmax(out_logits)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out_logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# multi layer lstm

In [18]:
import tensorflow as tf
from tensorflow.contrib import rnn

learning_rate = 0.001
training_iters = 100000
batch_size = BATCHSIZE
display_step = 10
nr_layers = 4

tf.reset_default_graph()

# Network Parameters
n_input = nr_joints*2 # MNIST data input (img shape: 28*28)
n_steps = N # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 12 # MNIST total classes (0-9 digits)

# tf Graph input
x_ = tf.placeholder("float", [None, n_steps, nr_joints, 2])
x = tf.reshape(x_, (-1, N, nr_joints*2))
y = tf.placeholder("float", [None, n_classes])

"""# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}"""


def RNN(x):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)
    
    # Define a lstm cell with tensorflow
    #lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
    def lstm_cell():
          return rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(nr_layers)])


    # Get lstm cell output
    """with tf.variable_scope("myrnn") as scope:
        for i in range(n_steps-1):
            if i > 0:
                scope.reuse_variables()
            output, state = lstm_cell(x[i], state)"""

    outputs, states = rnn.static_rnn(stacked_lstm, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.layers.dense(outputs[-1], n_classes)   #tf.matmul(outputs[-1], weights['out']) + biases['out']

out_logits = RNN(x)
out = tf.nn.softmax(out_logits)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out_logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# training

In [19]:
def batches(x, y, batchsize=32):
    permute = np.random.permutation(len(x))
    for i in range(0, len(x)-batchsize, batchsize):
        indices = permute[i:i+batchsize]
        yield x[indices], y[indices]

def balanced_batches(x, y, batchsize=32):
    for j in range(200):
        liste=np.zeros((nr_classes, ex_per_class))
        for i in range(nr_classes):
            # print(j, i, np.random.choice(index_liste[i][0], ex_per_class))
            liste[i] = np.random.choice(index_liste[i][0], ex_per_class, replace=False)
        liste = liste.flatten().astype(int)
        yield x[liste], y[liste]

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Run session for EPOCH epochs
for epoch in range(EPOCHS + 1):
    for batch_x, batch_t in balanced_batches(train_x, train_t, BATCHSIZE):
        sess.run(optimizer, {x_: batch_x, y: batch_t})
    # print("Loss test: ", sess.run(loss, {x: test_x, y: test_t}))
    #print("Loss train: ", sess.run(loss, {x: train_x, y: train_t}))

    #Test Accuracy
    loss_test, out_test = sess.run([loss,out], {x_: test_x, y: test_t})
    print("Loss test", loss_test)
    pitches_test = decode_one_hot(out_test, unique)
    print("Accuracy test: ", np.sum(np.asarray(labels_string_test)==pitches_test)/len_test)
    
    #Train Accuracy
    out_train = sess.run(out, {x_: train_x, y: train_t})
    pitches_train = decode_one_hot(out_train, unique)
    print("Accuracy train: ", np.sum(np.asarray(labels_string_train)==pitches_train)/SEP)

print(pitches_test)

Loss test 2.09129
Accuracy test:  0.0134831460674
Accuracy train:  0.0189762796504
Loss test 2.02599
Accuracy test:  0.0
Accuracy train:  0.0


KeyboardInterrupt: 

In [43]:
import numpy as np

out = np.array(["ine", "ine", "bdeu", "hdba", "nina", "nina"])
ground_truth = np.array(["ine","ine", "bdeu", "ine", "nina", "halo"])

same = out[np.where(out==ground_truth)]

right_frequency = sp.stats.itemfreq(same)
total_frequency = sp.stats.itemfreq(ground_truth)
right_dict = dict(zip(right_frequency[:,0], right_frequency[:,1]))
total_dict = dict(zip(total_frequency[:,0], total_frequency[:,1]))

acc = right_dict
for types in right_dict.keys():
    acc[types] = (int(right_dict[types])/float(total_dict[types]))
    
print(acc)

{'ine': 0.6666666666666666, 'bdeu': 1.0, 'nina': 1.0}
